# Agenti Azure AI con supporto per il Model Context Protocol (MCP) - Python

Questo notebook dimostra come utilizzare gli agenti Azure AI con gli strumenti del Model Context Protocol (MCP) in Python. Mostra come creare un agente intelligente che può sfruttare server MCP esterni (come Microsoft Learn) per capacità avanzate utilizzando l'autenticazione senza chiavi.


## Installare i Pacchetti Python Necessari

Per prima cosa, dobbiamo installare i pacchetti Python necessari:
- **azure-ai-projects**: SDK principale per Azure AI Projects
- **azure-ai-agents**: SDK di Azure AI Agents per creare e gestire agenti
- **azure-identity**: Fornisce l'autenticazione senza chiavi utilizzando DefaultAzureCredential
- **mcp**: Implementazione del Model Context Protocol per Python


## Vantaggi dell'Autenticazione Senza Chiavi

Questo notebook dimostra l'**autenticazione senza chiavi**, che offre diversi vantaggi:
- ✅ **Nessuna gestione di chiavi API** - Utilizza l'autenticazione basata sull'identità di Azure
- ✅ **Maggiore sicurezza** - Nessun segreto memorizzato nel codice o nei file di configurazione
- ✅ **Rotazione automatica delle credenziali** - Azure gestisce il ciclo di vita delle credenziali
- ✅ **Controllo degli accessi basato sui ruoli** - Utilizza Azure RBAC per permessi dettagliati
- ✅ **Supporto multi-ambiente** - Funziona senza problemi sia in sviluppo che in produzione

Il `DefaultAzureCredential` seleziona automaticamente la migliore fonte di credenziali disponibile:
1. **Managed Identity** (quando eseguito in Azure)
2. Credenziali di **Azure CLI** (durante lo sviluppo locale)
3. Credenziali di **Visual Studio**
4. **Variabili d'ambiente** (se configurate)
5. Autenticazione tramite **browser interattivo** (come ultima risorsa)


## Configurazione dell'autenticazione senza chiavi

**Prerequisiti per l'autenticazione senza chiavi:**

### Per lo sviluppo locale:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Per gli ambienti Azure:
- Abilita l'**Identità Gestita Assegnata dal Sistema** sulla tua risorsa Azure
- Assegna i ruoli **RBAC** appropriati all'identità gestita:
  - `Cognitive Services OpenAI User` per l'accesso ad Azure OpenAI
  - `AI Developer` per l'accesso ai progetti Azure AI

### Variabili d'ambiente (Opzionale):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nessuna chiave API o stringa di connessione necessaria!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importa le Librerie Necessarie

Importa i moduli Python necessari:
- **os, time**: Librerie standard di Python per variabili d'ambiente e ritardi
- **AIProjectClient**: Client principale per i progetti Azure AI
- **DefaultAzureCredential**: Autenticazione senza chiavi per i servizi Azure
- **Classi relative a MCP**: Per creare e gestire strumenti MCP e gestire approvazioni


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configurare le Impostazioni del Server MCP

Configura il server MCP utilizzando variabili di ambiente con valori predefiniti di fallback:
- **MCP_SERVER_URL**: L'URL del server MCP (predefinito: Microsoft Learn API)
- **MCP_SERVER_LABEL**: Un'etichetta per identificare il server MCP (predefinito: "mslearn")

Questo approccio consente una configurazione flessibile in diversi ambienti.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Creare un Client per Progetto Azure AI (Autenticazione Senza Chiave)

Inizializza il client per il progetto Azure AI utilizzando **l'autenticazione senza chiave**:
- **endpoint**: L'URL dell'endpoint del progetto Azure AI Foundry
- **credential**: Utilizza `DefaultAzureCredential()` per un'autenticazione sicura e senza chiave
- **Nessuna chiave API richiesta**: Rileva e utilizza automaticamente la credenziale migliore disponibile

**Flusso di Autenticazione:**
1. Verifica l'identità gestita (in ambienti Azure)
2. Passa alle credenziali di Azure CLI (per lo sviluppo locale)
3. Utilizza altre fonti di credenziali disponibili, se necessario

Questo approccio elimina la necessità di gestire chiavi API o stringhe di connessione nel codice.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Creare la Definizione dello Strumento MCP

Crea uno strumento MCP che si connette al server MCP di Microsoft Learn:
- **server_label**: Identificatore per il server MCP
- **server_url**: Endpoint URL del server MCP
- **allowed_tools**: Elenco opzionale per limitare quali strumenti possono essere utilizzati (un elenco vuoto consente tutti gli strumenti)

Questo strumento permetterà all'agente di accedere alla documentazione e alle risorse di Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Creare un Agente e Eseguire una Conversazione (Flusso Senza Chiavi)

Questa sezione completa illustra il flusso di lavoro **senza chiavi** per un agente:

1. **Creare un Agente AI**: Configura un agente con il modello GPT-4.1 nano e strumenti MCP
2. **Creare un Thread**: Stabilisci un thread di conversazione per la comunicazione
3. **Inviare un Messaggio**: Chiedi all'agente le differenze tra Azure OpenAI e OpenAI
4. **Gestire le Approvazioni degli Strumenti**: Approva automaticamente le chiamate agli strumenti MCP quando necessario
5. **Monitorare l'Esecuzione**: Tieni traccia dei progressi dell'agente e gestisci eventuali azioni richieste
6. **Mostrare i Risultati**: Visualizza i dettagli della conversazione e dell'utilizzo degli strumenti

**Caratteristiche Senza Chiavi:**
- ✅ **Nessun segreto hardcoded** - Tutta l'autenticazione gestita tramite identità Azure
- ✅ **Sicuro per impostazione predefinita** - Utilizza il controllo degli accessi basato sui ruoli
- ✅ **Distribuzione semplificata** - Non è richiesta la gestione delle credenziali
- ✅ **Audit-friendly** - Tutti gli accessi sono tracciati tramite identità Azure

L'agente utilizzerà gli strumenti MCP per accedere alle risorse di Microsoft Learn con piena sicurezza e senza gestione di chiavi API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
